# Sequence Models II
## 5.1 

### Assignment Solution - Sentiment on Amazon Product Reviews (Toys and Games)

In [14]:
!pip install tensorflow==2.0.0-beta0

In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/gdrive/My Drive/amazon_reviews/reviews_Toys_and_Games_5.json.gz')
#df = getDF('/content/gdrive/My Drive/amazon_reviews/reviews_Automotive_5.json.gz')

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5.0,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4.0,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5.0,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5.0,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5.0,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"


### Remove Special Characters from Review Text

In [0]:
import re

def strip_special_chars(st):
    my_pattern = '[A-Za-z0-9./ ]+'
    return ''.join(re.findall(my_pattern, st))

In [19]:
df['cleanReviewText'] = df['reviewText'].apply(strip_special_chars)
print(df['cleanReviewText'][0])

I needed a set of jumper cables for my new car and these had good reviews and were at a good price.  They have been used a few times already and do what they are supposed to  no complaints there.What I will say is that 12 feet really isnt an ideal length.  Sure if you pull up front bumper to front bumper they are plenty long but a lot of times you will be beside another car or cant get really close.  Because of this I would recommend something a little longer than 12.Great brand  get 16 version though.


### Convert Ratings to Sentiments

In [20]:
def sentiment(x):
    if x < 3:
        return 0
    else:
        return 1
df['sentiment'] = df['overall'].apply(lambda x: sentiment(x))
df[['cleanReviewText', 'overall', 'sentiment']].head()

,cleanReviewText,overall,sentiment
0,I needed a set of jumper cables for my new car...,5.0,1
1,These long cables work fine for my truck but t...,4.0,1
2,Cant comment much on these since they have not...,5.0,1
3,I absolutley love Amazon For the price of a s...,5.0,1
4,I purchased the 12 feet long cable set and the...,5.0,1


In [21]:
def word_sequence(x):
  return tf.keras.preprocessing.text.text_to_word_sequence(x, filters='!.', lower=True, split=' ')
df['sequence'] = df['cleanReviewText'].apply(lambda x: word_sequence(x))
df[['sequence', 'sentiment']].head()

,sequence,sentiment
0,"[i, needed, a, set, of, jumper, cables, for, m...",1
1,"[these, long, cables, work, fine, for, my, tru...",1
2,"[cant, comment, much, on, these, since, they, ...",1
3,"[i, absolutley, love, amazon, for, the, price,...",1
4,"[i, purchased, the, 12, feet, long, cable, set...",1


### Split data into features and labels

In [0]:
features = df['cleanReviewText']
labels = df['sentiment']

In [45]:
print(features[:3].to_numpy())

['I needed a set of jumper cables for my new car and these had good reviews and were at a good price.  They have been used a few times already and do what they are supposed to  no complaints there.What I will say is that 12 feet really isnt an ideal length.  Sure if you pull up front bumper to front bumper they are plenty long but a lot of times you will be beside another car or cant get really close.  Because of this I would recommend something a little longer than 12.Great brand  get 16 version though.'
 'These long cables work fine for my truck but the quality seems a little on the shabby side. For the money I was not expecting 200 dollar snapon jumper cables but these seem more like what you would see at a chinese knock off shop like harbor freight for 30 bucks.'
 'Cant comment much on these since they have not yet been used I will come back and update my review is I find any issues after first use ... but they are built solid nice tough big hard clamps and love having a long cable

### Embeddings require us to convert our text into a numerical representation. Let's use a layer from TensorFlow Hub to side-step this requirement

In [0]:
import tensorflow_hub as hub

### Instantiate an embedding with 50 dimensions

In [0]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", output_shape=[50], input_shape=[], dtype=tf.string)

### Let's create our model

In [32]:
model = keras.Sequential([
    hub_layer,
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 408       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 48,191,017
Trainable params: 417
Non-trainable params: 48,190,600
_________________________________________________________________
None


### Train the model

In [46]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x=features.to_numpy(), y=labels.to_numpy(), epochs=1, validation_split=0.3)

Train on 14331 samples, validate on 6142 samples
14331/14331 [==============================] - 50s 3ms/sample - loss: 0.2010 - accuracy: 0.9443 - val_loss: 0.2028 - val_accuracy: 0.9429
